In [1]:
import numpy  as np
import torch
from ase.units import kB

Read potential energies.

In [28]:
def cv(E,T):
    E = E.flatten()
    # eV**2 / ((eV/K) * K ** 2) -> eV/K
    return (np.mean((E - np.mean(E))**2))/(kB*T**2)

def CEA(obs, pot, T):

    #assuming obs is an array of shape ( N_Prop_dim, N_samples)
    #and pot is an array of shape ( N_samples, N_ens)
    # T is the temperature in K
    # returns N_ensemble weighted average obs

    beta = 1/(kB*T)
    w_obs = []
    
    obs_mean = np.mean(obs, axis=1) # (N, 1)
    pot_mean = np.mean(pot, axis=1) 


    #print(obs_mean.shape)
    #print(pot_mean.shape)

    for i in range(pot.shape[-1]):
        #print(np.mean(obs * (pot[:,i] - pot_mean),axis=1).shape)
        #print((pot[:,i] - pot_mean).shape)
        #print((np.mean(pot[:,i] - pot_mean)).shape)

        w_obs_i = obs_mean - beta * (np.mean(obs * (pot[:,i] - pot_mean),axis=1) - obs_mean * np.mean(pot[:,i] - pot_mean))
        w_obs.append(w_obs_i)

    return w_obs 


In [29]:
Temp = 300
N_H2O = 256
DROP_N = 200

with open(f'../../Atomistic_experiments/trajectories_H2O/DFT_size_extensive_MD_{Temp}K/run_{N_H2O}/BPNN-H2O-md.committee_pot_0', 'r') as file:
    lines = file.readlines()

dat = []
for line in lines[1::2]:
    tmp = []
    for s in line.split():
        tmp.append(float(s))
    dat.append(tmp)

dat = np.array(dat)
E_POTS = dat

U_mean = np.mean(E_POTS[DROP_N:],axis=1).reshape(-1,1)

#reweighting
out_energy = np.stack(CEA(U_mean.T, E_POTS[DROP_N:], Temp)) # Nsamples, Ndim
out_energy_squared = np.stack(CEA(U_mean.T**2, E_POTS[DROP_N:], Temp)) #Nsamples, Ndim

heat_coeffs = []


for E_i, E_i_sq in zip(out_energy, out_energy_squared):
    cv_i = ((E_i_sq - E_i**2)/(kB*Temp**2))/(N_H2O)/kB  + 9/2
    heat_coeffs.append(float(cv_i))

print("heat_coeff: {:.2f} pm {:.2f}".format(np.mean(heat_coeffs),np.std(heat_coeffs)))

heat_coeff: -0.80 pm 20.88


/var/folders/7y/yhl65v0j5m3g8zj_xwkzq44w0000gn/T/ipykernel_43752/2689354620.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  heat_coeffs.append(float(cv_i))


In [30]:
heat_cap_sampling = []

for block_i in E_POTS.T:
        heat_cap_sampling.append((cv(block_i[DROP_N:],Temp)/N_H2O)/kB)

print(np.mean(heat_cap_sampling) + 9/2)
print(np.std(heat_cap_sampling))

14.641328236871388
0.47356687226161515
